In [25]:
import pickle
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px 
import glob
import numpy as np

In [26]:
NODE_COUNT=4

In [27]:
results = glob.glob(f"./../results/NO_META_LEARNING*{NODE_COUNT}*/")
len(results)

7

In [28]:
results

['./../results/NO_META_LEARNING_MEDIAN_AGGREGATION_4_NODES/',
 './../results/NO_META_LEARNING_DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_4_NODES/',
 './../results/NO_META_LEARNING_WEIGHTED_AVERAGE_RECENCY_4_NODES/',
 './../results/NO_META_LEARNING_AVERAGE_AGGREGATION_4_NODES/',
 './../results/NO_META_LEARNING_NO_FEDERATED_LEARNING_4_NODES/',
 './../results/NO_META_LEARNING_FIXED_CLIPPED_AVERAGE_AGGREGATION_4_NODES/',
 './../results/NO_META_LEARNING_WEIGHTED_AVERAGE_PUNISHING_UPDATES_4_NODES/']

In [29]:
iter_dfs = [pickle.load(open(file +"/iter_df.pkl","rb")) for file in results]
training_dfs = [pickle.load(open(file +"/training_df.pkl","rb")) for file in results]
aggregation_methods=["_".join(file.split("_")[3:-2]) for file in results]

In [30]:
aggregation_methods

['MEDIAN_AGGREGATION',
 'DYNAMIC_CLIPPED_AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_RECENCY',
 'AVERAGE_AGGREGATION',
 'NO_FEDERATED_LEARNING',
 'FIXED_CLIPPED_AVERAGE_AGGREGATION',
 'WEIGHTED_AVERAGE_PUNISHING_UPDATES']

In [31]:
iter_dfs[0]

,curr_score_0,iter_counter_0,DG_rotation_0,timestamp_0,curr_score_1,iter_counter_1,DG_rotation_1,timestamp_1,curr_score_2,iter_counter_2,DG_rotation_2,timestamp_2,curr_score_3,iter_counter_3,DG_rotation_3,timestamp_3
0,NaN,1,0,2024-02-19 20:38:25.502268,NaN,1,1,2024-02-19 20:38:25.239443,NaN,1,0,2024-02-19 20:38:24.824776,NaN,1,0,2024-02-19 20:38:25.155188
1,0.808000,2,0,2024-02-19 20:38:33.558320,0.810333,2,1,2024-02-19 20:38:36.562226,0.781000,2,0,2024-02-19 20:38:30.281562,0.802000,2,0,2024-02-19 20:38:35.672288
2,0.805667,3,0,2024-02-19 20:38:34.208063,0.805667,3,1,2024-02-19 20:38:37.201229,0.782000,3,0,2024-02-19 20:38:30.949325,0.804667,3,0,2024-02-19 20:38:36.312650
3,0.808333,4,0,2024-02-19 20:38:34.862598,0.807667,4,2,2024-02-19 20:38:37.848856,0.782333,4,0,2024-02-19 20:38:31.600858,0.801000,4,0,2024-02-19 20:38:36.953553
4,0.810333,5,0,2024-02-19 20:38:35.512342,0.810333,5,2,2024-02-19 20:38:38.491150,0.781667,5,0,2024-02-19 20:38:32.257881,0.803333,5,0,2024-02-19 20:38:37.594102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.676000,996,197,2024-02-19 20:52:36.229398,0.835000,996,280,2024-02-19 20:53:27.298017,0.767000,996,576,2024-02-19 20:59:02.101137,0.741000,996,508,2024-02-19 20:57:43.325836
996,0.675000,997,197,2024-02-19 20:52:36.880576,0.834667,997,280,2024-02-19 20:53:27.948727,0.745333,997,577,2024-02-19 20:59:02.745362,0.717333,997,509,2024-02-19 20:57:43.983799
997,0.681667,998,198,2024-02-19 20:52:37.530342,0.825000,998,281,2024-02-19 20:53:28.595754,0.733667,998,578,2024-02-19 20:59:03.392283,0.795667,998,510,2024-02-19 20:57:54.624603
998,0.678333,999,198,2024-02-19 20:52:38.174812,0.815333,999,282,2024-02-19 20:53:29.243324,0.730667,999,578,2024-02-19 20:59:04.037980,0.803000,999,510,2024-02-19 20:57:55.267963


In [32]:
scores=[]
for aggregation_method,iter_df in zip(aggregation_methods,iter_dfs):
    rel_columns= [c for c in iter_df.columns if "curr_score_" in c]
    rename_dict = dict()
    for rel_column in rel_columns:
        rename_dict[rel_column] = "_".join([aggregation_method,rel_column])
    rel_df = iter_df[rel_columns]
    scores.append(rel_df.rename(columns=rename_dict, inplace=False))

In [33]:
score_df=pd.concat(scores,axis=1)
score_df

,MEDIAN_AGGREGATION_curr_score_0,MEDIAN_AGGREGATION_curr_score_1,MEDIAN_AGGREGATION_curr_score_2,MEDIAN_AGGREGATION_curr_score_3,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_0,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_1,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_2,DYNAMIC_CLIPPED_AVERAGE_AGGREGATION_curr_score_3,WEIGHTED_AVERAGE_RECENCY_curr_score_0,WEIGHTED_AVERAGE_RECENCY_curr_score_1,...,NO_FEDERATED_LEARNING_curr_score_2,NO_FEDERATED_LEARNING_curr_score_3,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_0,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_1,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_2,FIXED_CLIPPED_AVERAGE_AGGREGATION_curr_score_3,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_0,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_1,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_2,WEIGHTED_AVERAGE_PUNISHING_UPDATES_curr_score_3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.808000,0.810333,0.781000,0.802000,0.808333,0.810333,0.808000,0.790000,0.780667,0.808000,...,0.809667,0.773333,0.808000,0.810333,0.781000,0.809667,0.786000,0.808000,0.809333,0.810333
2,0.805667,0.805667,0.782000,0.804667,0.806333,0.805667,0.805667,0.794000,0.782333,0.805667,...,0.806333,0.777333,0.805667,0.805667,0.784333,0.808000,0.788000,0.805667,0.806333,0.805667
3,0.808333,0.807667,0.782333,0.801000,0.803667,0.807667,0.808333,0.793667,0.773667,0.808333,...,0.798333,0.777667,0.808333,0.807667,0.775667,0.805333,0.780667,0.808333,0.792667,0.807667
4,0.810333,0.810333,0.781667,0.803333,0.803000,0.810333,0.810333,0.786667,0.774000,0.810333,...,0.803000,0.775333,0.810333,0.810333,0.761333,0.810000,0.778667,0.810333,0.775667,0.810333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.676000,0.835000,0.767000,0.741000,0.800667,0.769000,0.662000,0.809000,0.750333,0.794667,...,0.895000,0.828333,NaN,NaN,NaN,NaN,0.750667,0.764333,0.818000,0.755000
996,0.675000,0.834667,0.745333,0.717333,0.802667,0.769000,0.660667,0.808667,0.753000,0.801000,...,0.889333,0.831667,NaN,NaN,NaN,NaN,0.748000,0.764000,0.818333,0.756000
997,0.681667,0.825000,0.733667,0.795667,0.806333,0.752000,0.659667,0.805333,0.752667,0.776667,...,0.889333,0.832333,NaN,NaN,NaN,NaN,0.752000,0.754667,0.820667,0.735000
998,0.678333,0.815333,0.730667,0.803000,0.794000,0.732333,0.661000,0.811000,0.731667,0.784667,...,0.890667,0.829333,NaN,NaN,NaN,NaN,0.749333,0.752333,0.801333,0.711333


In [34]:
average_score_dict=dict()
for aggregation_method in aggregation_methods:
    rel_col=[c for c in score_df.columns if aggregation_method == c[:-13]]
    result_dict=dict()
    by_node_result_average_dict=dict()
    for col in rel_col:
        by_node_result_average_dict[col.split("_")[-1]]=score_df[col].mean()
    result_dict["by_node_results"]=score_df[rel_col].mean(axis=0)
    result_dict["total_average"]=score_df[rel_col].mean().mean()
    result_dict["average_in_time"]= score_df[rel_col].mean(axis=1)
    average_score_dict[aggregation_method]=result_dict

In [35]:
len("_curr-score_0")

13

In [36]:
aggregation_names=[]
aggregation_averages=[]
by_node_averages=[]
average_in_time=[]
for key, val in average_score_dict.items():
    aggregation_names.append(key)
    aggregation_averages.append(val["total_average"])
    by_node_averages.append(val["by_node_results"].values)
    average_in_time.append(val["average_in_time"])
sorted_tuples=sorted(zip(aggregation_names,aggregation_averages,by_node_averages,average_in_time),key=lambda x: x[1])
aggregation_names = [val[0] for val in sorted_tuples]
aggregation_averages = [val[1] for val in sorted_tuples]
by_node_averages = [val[2] for val in sorted_tuples]
average_in_time = [val[3] for val in sorted_tuples]

In [37]:
n_nodes=len(by_node_averages[0])
n_nodes

4

In [38]:
by_node_averages

[array([0.65933805, 0.62910842, 0.55717576, 0.6046835 ]),
 array([0.72084651, 0.72961361, 0.72897497, 0.75361562]),
 array([0.7534044 , 0.75799066, 0.75411078, 0.76342643]),
 array([0.75462796, 0.78511578, 0.7372983 , 0.7787671 ]),
 array([0.77586553, 0.78394828, 0.75614414, 0.78267334]),
 array([0.7754361 , 0.78292025, 0.77300133, 0.77687287]),
 array([0.81372439, 0.81023557, 0.81467568, 0.7691368 ])]

In [39]:
processed_aggregation_names=[" ".join(name.split("_")[:-1]) for name in aggregation_names]
processed_aggregation_names

['FIXED CLIPPED AVERAGE',
 'WEIGHTED AVERAGE',
 'MEDIAN',
 'WEIGHTED AVERAGE PUNISHING',
 'DYNAMIC CLIPPED AVERAGE',
 'AVERAGE',
 'NO FEDERATED']

In [40]:
aggregation_method

'WEIGHTED_AVERAGE_PUNISHING_UPDATES'

In [41]:
name_mapping={"FIXED_CLIPPED_AVERAGE_AGGREGATION": "Fixed clip",
 "MEDIAN_AGGREGATION": "Median",
 "DYNAMIC_CLIPPED_AVERAGE_AGGREGATION": "Dynamic clip",
 "WEIGHTED_AVERAGE_PUNISHING_UPDATES": "Punish updates",
 "AVERAGE_AGGREGATION": "Simple average",
 "WEIGHTED_AVERAGE_RECENCY": "Receny weighted",
 "WEIGHTED_AVERAGE_PERFORMANCE_SCORES": "Performance weighted",
 "NO_FEDERATED_LEARNING": "No federated learning"}
processed_aggregation_names = [name_mapping[name] for name in aggregation_names]
processed_aggregation_names

['Fixed clip',
 'Receny weighted',
 'Median',
 'Punish updates',
 'Dynamic clip',
 'Simple average',
 'No federated learning']

In [42]:
fig=go.Figure()
for node in range(n_nodes):
    fig.add_trace(go.Bar(x=processed_aggregation_names, y=[val[node] for val in by_node_averages],name=f"Node {node}"))
fig.add_trace(go.Bar(x=processed_aggregation_names, y=aggregation_averages,name="Average"))
fig.update_layout(template="none", title=f"Accuracy score by aggregation methods for {NODE_COUNT} nodes",
                  width=800,height=400)
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Average_score_by_aggregation_{NODE_COUNT}_nodes.svg")
fig.show()

In [43]:
fig=go.Figure()
for idx,name in enumerate(processed_aggregation_names):
    fig.add_trace(go.Scatter(y=average_in_time[idx][1:] ,name=name))
fig.update_layout(template="none", title=f"Accuracy score by aggregation methods for {NODE_COUNT} nodes",
                  width=800,height=600,legend=dict( orientation="h",y=1.2),margin=dict(t=150,r=10))
fig.update_xaxes(title="Aggregation methods")
fig.update_yaxes(title="Accuracy score")
fig.write_image(f"./../figs/Performance_in_time_by_aggregation_{NODE_COUNT}_nodes.svg")
fig.show()

In [44]:
training_dfs[0]

,batch_size_mean_0,batch_size_std_0,learning_rate_mean_0,learning_rate_std_0,nesterov_mean_0,nesterov_std_0,momentum_mean_0,momentum_std_0,n_runs_mean_0,n_runs_std_0,...,n_runs_3,n_epochs_3,weights_3,iter_count_3,pre_training_score_3,scores_3,best_score_3,best_hyper_params_3,best_model_weights_3,timestamp_3
0,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,9.0,5.0,"[[[0.46430619060993195, -1.4068997502326965, 1...",485.0,0.400000,"[0.76, 0.8016666666666666, 0.7766666666666666,...",0.833333,"{'learning_rate': 0.000615824175084398, 'batch...","[[[0.3973318338394165, -1.2266579866409302, 1....",2024-02-19 20:47:43.360541
1,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,10.0,9.0,"[[[0.1167132705450058, -1.6149297952651978, 0....",218.0,0.475000,"[0.5216666666666666, 0.5283333333333333, 0.521...",0.605000,"{'learning_rate': 0.00019156247726025895, 'bat...","[[[-0.2709799110889435, -1.8446491956710815, 0...",2024-02-19 20:43:00.132733
2,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,9.0,8.0,"[[[0.46430619060993195, -1.439466655254364, 1....",473.0,0.388333,"[0.82, 0.8166666666666667, 0.8266666666666667,...",0.835000,"{'learning_rate': 0.0011038790956055071, 'batc...","[[[0.3165771961212158, -1.0261039733886719, 1....",2024-02-19 20:47:27.701677
3,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,7.0,10.0,"[[[0.8170947432518005, -1.4809311628341675, 1....",812.0,0.435000,"[0.7916666666666666, 0.7966666666666666, 0.79,...",0.796667,"{'learning_rate': 0.000497873885189525, 'batch...","[[[0.9244426488876343, -1.8189910650253296, 1....",2024-02-19 20:53:59.681011
4,64.0,20.0,-3.0,1.0,0.5,0.3,0.5,0.3,10.0,3.0,...,10.0,10.0,"[[[-0.04199022985994816, -1.622679352760315, 0...",322.0,0.471667,"[0.7666666666666667, 0.7733333333333333, 0.763...",0.810000,"{'learning_rate': 0.007326223904529604, 'batch...","[[[0.27752694487571716, -1.9188294410705566, 0...",2024-02-19 20:44:26.270310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,None,None,NaT
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,None,None,NaT
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,None,None,NaT
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,None,None,NaT


In [45]:
trainig_scores=[]
for aggregation_method,training_df in zip(aggregation_methods,training_dfs):
    rel_columns= [c for c in training_df.columns if "score" in c]
    rename_dict = dict()
    for rel_column in rel_columns:
        rename_dict[rel_column] = "_".join([aggregation_method,rel_column])
    rel_df = training_df[rel_columns]
    trainig_scores.append(rel_df.rename(columns=rename_dict, inplace=False))

In [46]:
training_scores_df=pd.concat(trainig_scores,axis=1)
training_scores_df

,MEDIAN_AGGREGATION_pre_training_score_0,MEDIAN_AGGREGATION_scores_0,MEDIAN_AGGREGATION_best_score_0,MEDIAN_AGGREGATION_pre_training_score_1,MEDIAN_AGGREGATION_scores_1,MEDIAN_AGGREGATION_best_score_1,MEDIAN_AGGREGATION_pre_training_score_2,MEDIAN_AGGREGATION_scores_2,MEDIAN_AGGREGATION_best_score_2,MEDIAN_AGGREGATION_pre_training_score_3,...,WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_0,WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_1,WEIGHTED_AVERAGE_PUNISHING_UPDATES_scores_1,WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_1,WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_2,WEIGHTED_AVERAGE_PUNISHING_UPDATES_scores_2,WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_2,WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_3,WEIGHTED_AVERAGE_PUNISHING_UPDATES_scores_3,WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_3
0,0.505000,"[0.85, 0.8383333333333334, 0.845, 0.8266666666...",0.855000,0.511667,"[0.7833333333333333, 0.7666666666666667, 0.776...",0.786667,0.411667,"[0.6466666666666666, 0.6466666666666666, 0.781...",0.781667,0.400000,...,0.625000,0.753333,"[0.775, 0.7883333333333333, 0.77, 0.7883333333...",0.788333,0.505000,"[0.81, 0.7983333333333333, 0.7783333333333333,...",0.810000,0.675000,"[0.8133333333333334, 0.7883333333333333, 0.803...",0.813333
1,0.780000,"[0.795, 0.8066666666666666, 0.8183333333333334...",0.818333,0.748333,"[0.8133333333333334, 0.82, 0.82, 0.805, 0.8233...",0.823333,0.678333,"[0.7833333333333333, 0.7983333333333333, 0.796...",0.806667,0.475000,...,0.786667,0.701667,"[0.7183333333333334, 0.6666666666666666, 0.763...",0.790000,0.488333,"[0.8, 0.8083333333333333, 0.815, 0.81666666666...",0.821667,0.688333,"[0.7966666666666666, 0.7966666666666666, 0.8, ...",0.808333
2,0.708333,"[0.815, 0.8233333333333334, 0.83, 0.8216666666...",0.835000,0.423333,"[0.8233333333333334, 0.81, 0.8216666666666667,...",0.845000,0.476667,"[0.8166666666666667, 0.7983333333333333, 0.815...",0.825000,0.388333,...,0.861667,0.490000,"[0.7933333333333333, 0.7866666666666666, 0.783...",0.793333,0.496667,"[0.8033333333333333, 0.8066666666666666, 0.805...",0.838333,0.665000,"[0.8033333333333333, 0.7933333333333333, 0.796...",0.806667
3,0.606667,"[0.82, 0.8483333333333334, 0.825, 0.815, 0.815...",0.848333,0.551667,"[0.8233333333333334, 0.8316666666666667, 0.818...",0.836667,0.513333,"[0.82, 0.8183333333333334, 0.8433333333333334,...",0.843333,0.435000,...,0.818333,0.500000,"[0.8, 0.7966666666666666, 0.79]",0.800000,0.680000,"[0.8083333333333333, 0.715, 0.81, 0.8066666666...",0.811667,0.603333,"[0.815, 0.8116666666666666, 0.8133333333333334...",0.815000
4,0.790000,"[0.795, 0.785, 0.8016666666666666, 0.775, 0.79...",0.801667,0.486667,"[0.635, 0.7316666666666667, 0.7183333333333334...",0.731667,0.506667,"[0.7933333333333333, 0.8566666666666667, 0.778...",0.856667,0.471667,...,0.573333,0.528333,"[0.8233333333333334, 0.8316666666666667, 0.823...",0.835000,0.521667,"[0.78, 0.8133333333333334, 0.7866666666666666,...",0.818333,0.418333,"[0.765, 0.7633333333333333, 0.7683333333333333...",0.780000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
len("_scores_0")

9

In [54]:
average_score_dict=dict()
for aggregation_method in aggregation_methods:
    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-21]]
    sub_filtered_rel_cols=[c for c in rel_col if "pre_training" in c]
    result_dict=dict()
    result_dict["by_node_results_pre_training"]=training_scores_df[sub_filtered_rel_cols].mean(axis=0)
    result_dict["total_average_pre_training"]=training_scores_df[sub_filtered_rel_cols].mean().mean()

    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-13]]
    sub_filtered_rel_cols=[c for c in rel_col if "best_score" in c]
    result_dict["by_node_results_best_score"]=training_scores_df[sub_filtered_rel_cols].mean(axis=0)
    result_dict["total_average_best_score"]=training_scores_df[sub_filtered_rel_cols].mean().mean()

    rel_col=[c for c in training_scores_df.columns if aggregation_method == c[:-9]]
    sub_filtered_rel_cols=[c for c in rel_col if "scores" in c]
    by_node_result_average_dict=dict()
    by_node_result_std_dict=dict()
    for col in sub_filtered_rel_cols:
        by_node_result_average_dict[col.split("_")[-1]]=np.mean(np.concatenate(training_scores_df[col].dropna().values))
        by_node_result_std_dict[col.split("_")[-1]]=np.std(np.concatenate(training_scores_df[col].dropna().values))
    result_dict["by_node_results_average_scores"]=by_node_result_average_dict
    result_dict["total_average_average_scores"]=np.mean([val for val in by_node_result_average_dict.values()])
    result_dict["by_node_results_std_scores"]=by_node_result_std_dict
    result_dict["total_average_std_scores"]=np.mean([val for val in by_node_result_std_dict.values()])
    average_score_dict[aggregation_method]=result_dict

In [49]:
for col in sub_filtered_rel_cols:
    by_node_result_average_dict[col.split("_")[-1]]=np.mean(np.concatenate(training_scores_df[col].dropna().values))

In [55]:
result_dict

{'by_node_results_pre_training': WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_0    0.564037
 WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_1    0.562885
 WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_2    0.540144
 WEIGHTED_AVERAGE_PUNISHING_UPDATES_pre_training_score_3    0.555811
 dtype: float64,
 'total_average_pre_training': 0.5557190353483457,
 'by_node_results_best_score': WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_0    0.788444
 WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_1    0.809359
 WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_2    0.799253
 WEIGHTED_AVERAGE_PUNISHING_UPDATES_best_score_3    0.806396
 dtype: float64,
 'total_average_best_score': 0.8008631721907583,
 'by_node_results_average_scores': {'0': 0.7656631762652706,
  '1': 0.7962651515151514,
  '2': 0.7535425101214575,
  '3': 0.7857836990595612},
 'total_average_average_scores': 0.7753136342403601,
 'by_node_results_std_scores': {'0': 0.07083169171306859,
  '1': 0.03622081406376297,
  

In [105]:
[(aggr,average_score_dict[aggr]["total_average_pre_training"]) for aggr in average_score_dict.keys()]

[('MEDIAN_AGGREGATION', 0.5562703423120089),
 ('DYNAMIC_CLIPPED_AVERAGE_AGGREGATION', 0.5791376050420167),
 ('WEIGHTED_AVERAGE_RECENCY', 0.5939859286626528),
 ('AVERAGE_AGGREGATION', 0.5395077045614679),
 ('NO_FEDERATED_LEARNING', 0.7358495069368464),
 ('FIXED_CLIPPED_AVERAGE_AGGREGATION', 0.5021062497953289),
 ('WEIGHTED_AVERAGE_PUNISHING_UPDATES', 0.5557190353483457)]

In [84]:
aggregation_names=[]
best_scores=[]
pre_training_scores=[]
average_scores=[]
std_scores=[]
for key, val in average_score_dict.items():
    aggregation_names.append(key)
    aggregation_averages.append(val["total_average"])
    by_node_averages.append(val["by_node_results"].values)
    average_in_time.append(val["average_in_time"])
sorted_tuples=sorted(zip(aggregation_names,aggregation_averages,by_node_averages,average_in_time),key=lambda x: x[1])
aggregation_names = [val[0] for val in sorted_tuples]
aggregation_averages = [val[1] for val in sorted_tuples]
by_node_averages = [val[2] for val in sorted_tuples]
average_in_time = [val[3] for val in sorted_tuples]

KeyError: 'by_node_results'